# STG_TEMP_MAX

## Setup

In [ ]:
# If necessary:
import os
os.remove('snorkel.db')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

## Parsing

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

### Define Parser

In [ ]:
import os
from snorkel.parser import CorpusParser, HTMLParser, OmniParser
from snorkel.utils import get_ORM_instance
from snorkel.queries import split_corpus

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
doc_parser = HTMLParser(path=docs_path)
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=100) 

### Run Parser

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

session.add(corpus)
session.commit()

### Split Corpus

In [ ]:
from snorkel.models import Corpus

corpus = get_ORM_instance(Corpus, session, 'Hardware')
split_corpus(session, corpus, train=0.8, development=0.2, test=0, seed=3)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ corpus');

## Extraction

In [ ]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ corpus snorkel.db');

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Define Matchers

In [ ]:
from snorkel.matchers import RegexMatchSpan, Union

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

temp_matcher = RegexMatchSpan(rgx=r'1[4-6]0', longest_match_only=False)
# temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]', longest_match_only=False)

### Define ContextSpaces

In [ ]:
import os
from collections import defaultdict
from hardware_utils import OmniNgramsPart, OmniNgramsTemp, get_gold_dict

# Make parts list
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
parts_by_doc = defaultdict(set)
for part in gold_parts:
    parts_by_doc[part[0]].add(part[1])
    
part_ngrams = OmniNgramsPart(parts_by_doc=parts_by_doc, n_max=3)
temp_ngrams = OmniNgramsTemp(n_max=2)

### Define Candidate Throttler

In [ ]:
from hardware_utils import PartThrottler
part_throttler = PartThrottler()

### Run CandidateExtractor

In [ ]:
from snorkel.models import Corpus
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
ce = CandidateExtractor(Part_Temp, [part_ngrams, temp_ngrams], [parts_matcher, temp_matcher], throttler=part_throttler)

for corpus_name in ['Hardware Training', 'Hardware Development']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

### Assess Recall

In [ ]:
from snorkel.models import Candidate
from hardware_utils import entity_level_total_recall

all_candidates = session.query(Candidate).all()
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(
    all_candidates, gold_file, 'stg_temp_max', relation=True)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ candidates');

## Gold Labels

In [ ]:
# If necessary
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ candidates snorkel.db');

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

In [ ]:
import os
from snorkel.models import CandidateSet
from hardware_utils import load_hardware_labels

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
for set_name in ['Training', 'Development']:
    candidate_set_name = 'Hardware %s Candidates' % set_name
    candidates = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name).one()
    label_set_name = 'Hardware %s Candidates -- Gold' % set_name
    annotation_key_name = 'Hardware %s Labels -- Gold' % set_name
    %time gold_candidates, annotation_key = load_hardware_labels(session,\
                           label_set_name, \
                           annotation_key_name, \
                           candidates, \
                           gold_file, \
                           'stg_temp_max')
    candidates_gold = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name + ' -- Gold').one()
    print "%d/%d Candidates in %s have positive Labels" % (
        len(candidates_gold), len(candidates), candidates)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ labels');

## Features

In [ ]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ labels snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Extract Features

In [ ]:
from snorkel.models import CandidateSet
from snorkel.fast_annotations import FeatureManager
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
dev   = get_ORM_instance(CandidateSet, session, 'Hardware Development Candidates')

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Train Features')
%time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

In [ ]:
# If necessary:
import os
os.system('cp snorkel.db snorkel.db\ featurized');

## LFs

In [ ]:
# If necessary
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ featurized snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()

### Define LFs

In [ ]:
from snorkel.fast_annotations import LabelManager
from snorkel.lf_helpers import *
import re
label_manager = LabelManager()

LFs = []

# POSITIVE

def LF_to_left(c):
    return 1 if 'to' in get_left_ngrams(c.temp, window=2) else 0
LFs.append(LF_to_left)

def LF_negative_left(c):
    return 1 if any([re.match(r'-\s*\d+', ngram) for ngram in get_left_ngrams(c.temp, window=4)]) else 0
LFs.append(LF_negative_left)

def LF_storage_row(c):
    return 1 if 'storage' in get_row_ngrams(c.temp) else 0
LFs.append(LF_storage_row)

def LF_operating_row(c):
    return 1 if 'operating' in get_row_ngrams(c.temp) else 0
LFs.append(LF_operating_row)

def LF_temperature_row(c):
    return 1 if 'temperature' in get_row_ngrams(c.temp) else 0
LFs.append(LF_temperature_row)

def LF_tstg_row(c):
    return 1 if overlap(
        ['tstg','stg','ts'], 
        list(get_row_ngrams(c.temp))) else 0
LFs.append(LF_tstg_row)

# NEGATIVE

def LF_not_temp_relevant(c):
    return -1 if not overlap(
        ['storage','temperature','tstg','ts','stg'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_not_temp_relevant)

def LF_temp_outside_table(c):
    return -1 if c.temp.parent.row is None else 0
LFs.append(LF_temp_outside_table)

def LF_too_many_numbers_row(c):
    num_numbers = list(get_row_ngrams(c.temp, attrib="ner_tags")).count('number')
    return -1 if num_numbers >= 3 else 0
LFs.append(LF_too_many_numbers_row)

def LF_collector_aligned(c):
    return -1 if overlap(
        ['collector', 'collector-current', 'collector-base', 'collector-emitter'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_collector_aligned)

def LF_current_aligned(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if overlap(
        ['current', 'dc', 'ic'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_current_aligned)

def LF_voltage_row_temp(c):
    ngrams = get_aligned_ngrams(c.temp)
    return -1 if overlap(
        ['voltage', 'cbo', 'ceo', 'ebo', 'v'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_voltage_row_temp)

def LF_voltage_row_part(c):
    ngrams = get_aligned_ngrams(c.part)
    return -1 if overlap(
        ['voltage', 'cbo', 'ceo', 'ebo', 'v'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_voltage_row_part)

def LF_typ_row(c):
    return -1 if overlap(
        ['typ', 'typ.'],
        list(get_row_ngrams(c.temp))) else 0
LFs.append(LF_typ_row)

def LF_test_condition_aligned(c):
    return -1 if overlap(
        ['test', 'condition'],
        list(get_aligned_ngrams(c.temp))) else 0
LFs.append(LF_test_condition_aligned)

def LF_complement_left_row(c):
    return -1 if 'complement' in get_row_ngrams(c.part) else 0
LFs.append(LF_complement_left_row)

### Apply LFs

In [ ]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

### Assess LF accuracy

In [ ]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()
%time L_train.lf_stats(train_gold)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

## Learn and Evaluate

In [1]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ features snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

# from snorkel.annotations import FeatureManager, LabelManager
# feature_manager = FeatureManager()
# %time F_train = feature_manager.load(session, train, 'Train Features')
# %time F_dev = feature_manager.load(session, dev, 'Train Features')

# label_manager = LabelManager()
# %time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 1min 10s, sys: 2.54 s, total: 1min 13s
Wall time: 1min 13s
CPU times: user 34.1 s, sys: 1.48 s, total: 35.6 s
Wall time: 35.8 s
CPU times: user 12.1 s, sys: 531 ms, total: 12.6 s
Wall time: 12.9 s


In [2]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=15000, rate=1e-2)
%time gen_model.save(session, 'Generative Params')
train_marginals = gen_model.marginals(L_train)

Training marginals (!= 0.5):	61982
Features:			16
Begin training for rate=0.01, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.428656
	Learning epoch = 250	Gradient mag. = 0.554836
	Learning epoch = 500	Gradient mag. = 0.624353
	Learning epoch = 750	Gradient mag. = 0.658952
	Learning epoch = 1000	Gradient mag. = 0.656587
	Learning epoch = 1250	Gradient mag. = 0.637571
	Learning epoch = 1500	Gradient mag. = 0.624727
	Learning epoch = 1750	Gradient mag. = 0.507007
	Learning epoch = 2000	Gradient mag. = 0.399121
	Learning epoch = 2250	Gradient mag. = 0.363564
	Learning epoch = 2500	Gradient mag. = 0.338219
	Learning epoch = 2750	Gradient mag. = 0.318006
	Learning epoch = 3000	Gradient mag. = 0.303478
	Learning epoch = 3250	Gradient mag. = 0.292682
	Learning epoch = 3500	Gradient mag. = 0.284690
	Learning epoch = 3750	Gradient mag. = 0.278917
	Learning epoch = 4000	Gradient mag. = 0.274916
	Learning epoch = 4250	Gradient mag. = 0.271489
	Learning epoch = 4500	Gradient mag. = 0.251673
	Lear

In [3]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=2000, rate=1e-4)
%time disc_model.save(session, "Discriminative Params")

Training marginals (!= 0.5):	61982
Features:			16200
Using gradient descent...
	Learning epoch = 0	Step size = 0.0001
	Loss = 42962.648545	Gradient magnitude = 71935.959755
	Learning epoch = 100	Step size = 9.04792147114e-05
	Loss = 1942.930014	Gradient magnitude = 123.435604
	Learning epoch = 200	Step size = 8.18648829479e-05
	Loss = 1864.325499	Gradient magnitude = 75.956231
	Learning epoch = 300	Step size = 7.40707032156e-05
	Loss = 1830.780246	Gradient magnitude = 54.361144
	Learning epoch = 400	Step size = 6.70185906007e-05
	Loss = 1816.781772	Gradient magnitude = 36.752384
	Learning epoch = 500	Step size = 6.06378944861e-05
	Loss = 1809.801557	Gradient magnitude = 30.332203
	Learning epoch = 600	Step size = 5.48646907485e-05
	Loss = 1805.221985	Gradient magnitude = 26.572696
	Learning epoch = 700	Step size = 4.96411413431e-05
	Loss = 1801.962949	Gradient magnitude = 23.956685
	Learning epoch = 800	Step size = 4.4914914861e-05
	Loss = 1799.530076	Gradient magnitude = 22.031956
	Le

In [4]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [5]:
dev_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [6]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Gold')

In [7]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, dev_gold)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.463169297431
Neg. class accuracy: 1.0
Corpus Precision 1.0
Corpus Recall    0.463
Corpus F1        0.633
----------------------------------------
TP: 2993 | FP: 0 | TN: 733 | FN: 3469

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.463169297431
Neg. class accuracy: 1.0
Corpus Precision 1.0
Corpus Recall    0.463
Corpus F1        0.633
----------------------------------------
TP: 2993 | FP: 0 | TN: 733 | FN: 3469



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [8]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_max')

Scoring on Entity-Level Gold Data
Corpus Precision 1.0
Corpus Recall    0.94
Corpus F1        0.969
----------------------------------------
TP: 156 | FP: 0 | FN: 10



In [9]:
from pprint import pprint
FN_list = sorted(list(FN))
pprint(FN_list)

[('SCSCS02324-1', 'MMBT3904', '150'),
 ('TKCGS00622-1', 'DTC114TE', '150'),
 ('TKCGS00622-1', 'DTC114YE', '150'),
 ('TKCGS00622-1', 'DTC123EE', '150'),
 ('TKCGS00622-1', 'DTC124EE', '150'),
 ('TKCGS00622-1', 'DTC124XE', '150'),
 ('TKCGS00622-1', 'DTC143EE', '150'),
 ('TKCGS00622-1', 'DTC143TE', '150'),
 ('TKCGS00622-1', 'DTC143ZE', '150'),
 ('TKCGS00622-1', 'DTC144EE', '150')]


In [16]:
from hardware_utils import entity_to_candidates

entity = FN_list[0]
print entity
print

matches = entity_to_candidates(entity, fn)
print "# Matches: %d" % len(matches)
candidate = matches[0]
print candidate
print

print disc_model.get_candidate_score(candidate, F_dev)
print
pprint(disc_model.get_candidate_feature_weights(candidate, F_dev))

('TKCGS00622-1', 'DTC114TE', '150')

# Matches: 2
Part_Temp(Span("DTC114TE", parent=104324, chars=[36,43], words=[4,4]), ImplicitSpan("150", parent=104347, words=[0,0], position=[0]))

-13.3157731116

[(AnnotationKey (TAB_e2_HTML_ANC_TAG_[p]), -1.0589626992406873),
 (AnnotationKey (TAB_e2_HTML_ANC_TAG_[html]), -1.0589626992406873),
 (AnnotationKey (TAB_e2_HTML_TAG_p), -1.0589626992406873),
 (AnnotationKey (TAB_e1_HTML_ANC_TAG_[html]), -1.0589626992406873),
 (AnnotationKey (TAB_e2_HTML_ANC_TAG_[body]), -1.0589626992406873),
 (AnnotationKey (TAB_e2_ROW_WORDS_[c]), 0.81309487283043),
 (AnnotationKey (TAB_e1_HTML_ANC_TAG_[body]), -0.7200657523626274),
 (AnnotationKey (TAB_e2_ROW_WORDS_[=]), -0.6055302504899801),
 (AnnotationKey (TAB_e2_COL_WORDS_[200]), 0.5831720973313892),
 (AnnotationKey (TAB_e2_HTML_ANC_TAG_[li]), 0.4844351273020427),
 (AnnotationKey (TAB_e2_ROW_WORDS_[v]), -0.46736275638108976),
 (AnnotationKey (TAB_e2_ROW_WORDS_[i]), -0.43367066687150707),
 (AnnotationKey (TAB_e2_HTML

In [20]:
from hardware_utils import part_error_analysis
part_error_analysis(candidate)

Doc: Document TKCGS00622-1
------------
Part:
Span("DTC114TE", parent=104324, chars=[36,43], words=[4,4])
Table: Table(Doc: TKCGS00622-1, Position: 3)
Row: 2
Col: 1
Phrase: Phrase(Doc: TKCGS00622-1, Table: 3, Row: 2, Col: 1, Position: 0, Text: DTC114EE DTC124EE DTC144EE DTC114YE DTC114TE DTC143TE DTC123EE DTC143EE DTC143ZE DTC124XE)
------------
Temp:
ImplicitSpan("150", parent=104347, words=[0,0], position=[0])
Table: Table(Doc: TKCGS00622-1, Position: 3)
Row: 2
Col: 4
Phrase: Phrase(Doc: TKCGS00622-1, Table: 3, Row: 2, Col: 4, Position: 0, Text: 150)


In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');